In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/College/F19/CS 4740/NLP_P4

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/College/F19/CS 4740/NLP_P4


In [0]:
# %pip install transformers

In [3]:
import numpy as np
import pandas as pd
import csv
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
import random
import os
import time
from tqdm import tqdm

train_data = pd.read_csv('./gpu_train.csv', encoding='latin-1')
dev_data = pd.read_csv('./gpu_dev.csv', encoding='latin-1')
test_data = pd.read_csv('./gpu_test.csv', encoding='latin-1')
data = [train_data, dev_data, test_data]
bert = torch.hub.load('huggingface/transformers', 'model', 'bert-base-uncased')
tokenizer = torch.hub.load('huggingface/transformers', 'tokenizer', 'bert-base-uncased')

EPOCHS = 1
LR = 0.001
DEVICE = torch.device("cuda:0")
NAME = 'bert1'
CURRENT = os.curdir
MODELS = os.path.join(CURRENT, 'experimental_models')
PATH = os.path.join(MODELS, NAME)

Using cache found in /root/.cache/torch/hub/huggingface_transformers_master


Using cache found in /root/.cache/torch/hub/huggingface_transformers_master


In [0]:
def train_and_classify(training_data, development_data):
    class BertnaryClassification(nn.Module):
        def __init__(self):
            super(BertnaryClassification, self).__init__()
            self.linear = nn.Linear(768, 2)
            self.softmax = nn.LogSoftmax(dim=0)
            self.criterion = nn.NLLLoss()
            self.optimizer = optim.Adam(self.parameters(), lr=LR)
            self.cuda(device=DEVICE)

        def compute_Loss(self, predicted_vector, gold_label):
            return self.criterion(predicted_vector, gold_label)

        def forward(self, input_vector):
            features = torch.mean(bert(input_vector)[0].squeeze(), dim=0).to(DEVICE)
            prediction = self.linear(features)
            return self.softmax(prediction)

    def setup(training_data, development_data):
        print('Initializing Setup')
        train_data = []
        for row in training_data.iterrows():
            ID = row[1][0]
            if row[1][7] == 1:
                pos_story = ' '.join(w for w in [word for sentence in row[1][1:6] for word in sentence])
                pos_story = torch.tensor([tokenizer.encode(pos_story, add_special_tokens=True)])
                pos = (ID, pos_story, 1)
                neg_story = ' '.join(w for w in [word for sentence in row[1][1:5] + [row[1][6]] for word in sentence])
                neg_story = torch.tensor([tokenizer.encode(neg_story, add_special_tokens=True)])
                neg = (ID, neg_story, 0)
            else:
                neg_story = ' '.join(w for w in [word for sentence in row[1][1:6] for word in sentence])
                neg_story = torch.tensor([tokenizer.encode(neg_story, add_special_tokens=True)])
                neg = (ID, neg_story, 0)
                pos_story = ' '.join(w for w in [word for sentence in row[1][1:5] + [row[1][6]] for word in sentence])
                pos_story = torch.tensor([tokenizer.encode(pos_story, add_special_tokens=True)])
                pos = (ID, pos_story, 1)
            train_data.append(pos)
            train_data.append(neg)

        dev_data = []
        for row in development_data.iterrows():
            ID = row[1][0]
            LABEL = row[1][7]
            story_1 = ' '.join(w for w in [word for sentence in row[1][1:6] for word in sentence])
            story_1 = torch.tensor([tokenizer.encode(story_1, add_special_tokens=True)], device=DEVICE, dtype=torch.float)
            story_2 = ' '.join(w for w in [word for sentence in row[1][1:5] + [row[1][6]] for word in sentence])
            story_2 = torch.tensor([tokenizer.encode(story_2, add_special_tokens=True)], device=DEVICE, dtype=torch.float)
            sample = (ID, story_1, story_2, LABEL)
            dev_data.append(sample)

        return train_data, dev_data

    def train(train_data):
        model = BertnaryClassification()
        for epoch in range(EPOCHS):
            model.train()
            model.optimizer.zero_grad()
            loss = None
            correct = 0
            total = 0
            random.shuffle(train_data)
            N = len(train_data)
            print('Training...')
            for index in tqdm(range(N)):
                model.optimizer.zero_grad()
                __, input_vector, gold_label = train_data[index]
                predicted_vector = model(input_vector)
                predicted_label = torch.argmax(predicted_vector)
                loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label], device=DEVICE))
                loss.backward()
                model.optimizer.step()
        return model

    def validate(dev_data, model):
        model.eval()
        model.optimizer.zero_grad()
        N = len(dev_data)
        labels = []
        print('Validating...')
        for index in tqdm(range(N)):
            __, input_1, input_2, __ = dev_data[index]
            prediction_1 = model(input_1)
            prediction_2 = model(input_2)
            prob_truthful_1 = prediction_1[1]
            prob_false_1 = prediction_1[0]
            prob_truthful_2 = prediction_2[1]
            prob_false_2 = prediction_2[0]
            probs = [prob_truthful_1, prob_false_1, prob_truthful_2, prob_false_2]
            max_index = probs.index(max(probs))
            if max_index == 0 or max_index == 3:
                predicted_label = 0
            if max_index == 1 or max_index == 2:
                predicted_label = 1
            labels.append(predicted_label)
    
    def accuracy(predictions, dev_data):
        correct = 0
        total = 0
        for i in range(len(predictions)):
            correct += int(predictions[i] == dev_data[i][3])
            total += 1
        return correct / total

    train_data, dev_data = setup(training_data, development_data)

    # create and train model
    model = train(train_data)

    # evaulate model
    predictions = validate(dev_data, model)
    acc = accuracy(predictions, dev_data)
    return 'accurary: ' + str(round(acc * 100, 2)) + '%'

In [0]:
train_and_classify(train_data, dev_data)

Initializing Setup


  0%|          | 0/2994 [00:00<?, ?it/s]

Training...


  6%|▋         | 191/2994 [08:21<1:55:20,  2.47s/it]